In [ ]:
import openai
import random
import pandas as pd

# Hàm để đánh giá thông tin bằng GPT
api_key = "sk-None-iOoyUPtcZK3npXWePKwaT3BlbkFJQrxLETpaB510hxCGTHWS" # "sk-None-0qLRg02ilC3Bg4I8Y2wpT3BlbkFJuqzBPXjDe3q3hVygANGG"
openai.api_key = api_key

In [ ]:
metrics = ['Return (Ann.) [%]',  'Volatility', 'Sharpe Ratio', 'Sortino Ratio',' Calmar Ratio', 'Win Rate', 'AvgTrade', 'GS', 'MDD', 'Tỉ lệ dự đoán gần đây']

# Prompt data for single metrics

In [ ]:
def evaluate_sharpe_scenario():
    scenarios = []
    
    for i in range(1, 80):  # Giả sử x từ 1 đến 100
        random_num =  round(random.uniform(-1, 1), 2) 
        prompt = f"""Giả sử có một mô hình dự đoán giá có chỉ số Generalization Score đo khả năng của mô hình trong việc hoạt động hiệu quả trên dữ liệu mới, chưa từng gặp trong quá trình huấn luyện, càng gần 0 càng tốt là '{random_num}', hãy đặt các tình huống hỏi đáp liên quan tới kết quả chỉ số đó. Trả về kết quả cho tôi bằng tiếng Việt với format: 
        'Question:...
         Choose:... 
         Reject:...'"""

        # Sử dụng ChatCompletion để gửi prompt tới GPT
        try:
            response = openai.ChatCompletion.create(
                model="gpt-4o-2024-08-06",  # Hoặc "gpt-4" nếu bạn muốn sử dụng mô hình GPT-4
                messages=[
                    {"role": "system", "content": "You are a financial analyst."},
                    {"role": "user", "content": prompt}
                ],
                max_tokens=200,  
                temperature=0.8  
            )
            
            # Lấy phản hồi từ GPT
            answer = response['choices'][0]['message']['content'].strip()
            scenarios.append((answer, random_num))
        
        except Exception as e:
            print(f"Error occurred: {e}")
            scenarios.append(("Error", random_num))

    return scenarios

# Gọi hàm để lấy các tình huống
results = evaluate_sharpe_scenario()

data = []
for entry, sharpe_ratio in results:
    # Tìm các phần Question, Choose, Reject
    questions = entry.split('Question:')[1:]  # Bỏ qua phần đầu
    for question in questions:
        try:
            question = question.strip()
            choose = question.split('Choose:')[1].split('Reject:')[0].strip()
            reject = question.split('Reject:')[1].strip()
            question_text = question.split('\n')[0].strip()  # Câu hỏi

            # Thêm vào dữ liệu
            data.append([question_text, choose, reject, sharpe_ratio])
        except IndexError:
            # Xử lý nếu một entry không đúng định dạng
            print(f"Warning: Entry không đúng định dạng:\n{question}")
            continue

# Tạo DataFrame
df = pd.DataFrame(data, columns=['Câu hỏi mẫu', 'Câu trả lời nên chọn', 'Câu trả lời nên từ chối', 'Value'])
df.to_csv('gs.csv', index=False)

# For concat

In [ ]:
df2= pd.read_csv('.csv')
df_new= pd.concat([df2,df], axis=0, ignore_index=True)
df_new.to_csv('1000.csv',index=False)

# Add context to data

## For sharpe

In [37]:
import pandas as pd
import random
data = pd.read_csv('prompt_data/sharpe.csv')

context = []
# Tạo các biến ngẫu nhiên
sharpe_array = data['Sharpe']
for i,_ in enumerate(sharpe_array):
    sharpe_ratio = sharpe_array[i]
    volatility_ann =  round(random.uniform(5, 35), 2)  
    return_ann = round(sharpe_ratio*volatility_ann, 2)  
    max_drawdown = round(random.uniform(-40, -10), 2)  
    sortino_ratio = round(random.uniform(0.5, 2.5), 2)  
    calmar_ratio = round((-1)*return_ann/max_drawdown, 2)
    win_rate = round(random.uniform(40, 70), 2)  
    avg_trade = round(random.uniform(-0.5, 2.5), 2) 
    generalization_score = round(random.uniform(-1, 1), 2) 
    recent_prediction_rate = random.randrange(0, 101, 10) 

    # Kết quả
    results = f"""Chỉ số của mô hình:
    Return (Ann.) [%]: {return_ann},
    Volatility (Ann.) [%]: {volatility_ann},
    Max. Drawdown [%]: {max_drawdown},
    Sharpe Ratio: {sharpe_ratio},
    Sortino Ratio: {sortino_ratio},
    Calmar Ratio: {calmar_ratio},
    Win Rate [%]: {win_rate},
    AvgTrade [%]: {avg_trade},
    Generalization Score: {generalization_score},
    Tỉ lệ dự đoán gần đây [%]: {recent_prediction_rate}.
    Các thông tin tài chính bổ sung: Sharpe Ratio là tỷ lệ so sánh giữa lợi nhuận thặng dư so với lãi suất phi rủi ro và độ biến động của lợi nhuận, người dùng có thể dùng chỉ số này để xem xét liệu mô hình có tin cậy không. Công thức tính là:
Sharpe Ratio = (Return − Risk-Free Rate)/ Volatility . Chỉ số này giúp đo lường hiệu quả của việc chấp nhận rủi ro để đạt được lợi nhuận. Tỷ lệ càng cao cho thấy mô hình càng hiệu quả trong việc tạo ra lợi nhuận với mức độ rủi ro chấp nhận được. Sharpe Ratio trên 1 được coi là tốt, trên 2 là rất tốt và trên 3 là xuất sắc. Chỉ số này dưới 1 thể hiện rằng mô hình có thể không đáng giá để chấp nhận rủi ro vì lợi nhuận không đủ bù đắp cho độ biến động
    """
    context.append(results)
data['Context']=context
data.to_csv('prompt_data/new_sharpe.csv', index=False)

## For Sortino

In [36]:
import pandas as pd
import random
data = pd.read_csv('prompt_data/sortino.csv')

context = []
# Tạo các biến ngẫu nhiên
sortino_array = data['Sortino']
for i,_ in enumerate(sortino_array):
    sharpe_ratio = round(random.uniform(-1.5, 3.5), 2)  
    volatility_ann =  round(random.uniform(5, 35), 2)  
    return_ann = round(sharpe_ratio*volatility_ann, 2)  
    max_drawdown = round(random.uniform(-40, -10), 2)  
    sortino_ratio = sortino_array[i]
    calmar_ratio = round((-1)*return_ann/max_drawdown, 2)
    win_rate = round(random.uniform(40, 70), 2)  
    avg_trade = round(random.uniform(-0.5, 2.5), 2) 
    generalization_score = round(random.uniform(-1, 1), 2) 
    recent_prediction_rate = random.randrange(0, 101, 10) 

    # Kết quả
    results = f"""Chỉ số của mô hình:
    Return (Ann.) [%]: {return_ann},
    Volatility (Ann.) [%]: {volatility_ann},
    Max. Drawdown [%]: {max_drawdown},
    Sharpe Ratio: {sharpe_ratio},
    Sortino Ratio: {sortino_ratio},
    Calmar Ratio: {calmar_ratio},
    Win Rate [%]: {win_rate},
    AvgTrade [%]: {avg_trade},
    Generalization Score: {generalization_score},
    Tỉ lệ dự đoán gần đây [%]: {recent_prediction_rate}.
    Các thông tin tài chính bổ sung: Sortino Ratio là một biến thể của Sharpe Ratio, chỉ tập trung vào phần rủi ro giảm (downside risk) thay vì toàn bộ độ biến động. Nó được tính bằng công thức 
Sortino Ratio  = (Return − Risk-Free Rate )/ Downside Deviation. Chỉ số này giúp đánh giá hiệu quả của mô hình khi đối mặt với rủi ro giảm, làm rõ sự khác biệt giữa rủi ro "tốt" (lợi nhuận cao) và rủi ro "xấu" (thua lỗ). Một Sortino Ratio trên 1 cũng được coi là tốt, và trên 2 là rất tích cực, cho thấy mô hình sinh lời hiệu quả mà không chịu quá nhiều rủi ro giảm. Chỉ số dưới 1 cho thấy rủi ro giảm có thể là vấn đề lớn.    """
    context.append(results)
data['Context']=context
data.to_csv('prompt_data/new_sortino.csv', index=False)

## For return

In [35]:
import pandas as pd
import random
data = pd.read_csv('prompt_data/return.csv')

context = []
# Tạo các biến ngẫu nhiên
return_array = data['Value']
for i,_ in enumerate(return_array):
    return_ann = return_array[i]
    volatility_ann =  round(random.uniform(5, 35), 2)  
    sharpe_ratio = round(return_ann/volatility_ann, 2)  
    max_drawdown = round(random.uniform(-40, -10), 2)  
    sortino_ratio = round(random.uniform(0.5, 2.5), 2)  
    calmar_ratio = round((-1)*return_ann/max_drawdown, 2)
    win_rate = round(random.uniform(40, 70), 2)  
    avg_trade = round(random.uniform(-0.5, 2.5), 2) 
    generalization_score = round(random.uniform(-1, 1), 2) 
    recent_prediction_rate = random.randrange(0, 101, 10) 

    # Kết quả
    results = f"""Chỉ số của mô hình:
    Return (Ann.) [%]: {return_ann},
    Volatility (Ann.) [%]: {volatility_ann},
    Max. Drawdown [%]: {max_drawdown},
    Sharpe Ratio: {sharpe_ratio},
    Sortino Ratio: {sortino_ratio},
    Calmar Ratio: {calmar_ratio},
    Win Rate [%]: {win_rate},
    AvgTrade [%]: {avg_trade},
    Generalization Score: {generalization_score},
    Tỉ lệ dự đoán gần đây [%]: {recent_prediction_rate}.
    Các thông tin tài chính bổ sung: Return (Ann.) [%]: Tỷ suất lợi nhuận hàng năm là chỉ số phản ánh mức lợi nhuận trung bình mà mô hình dự đoán tạo ra mỗi năm, tính theo phần trăm. Đây là một thước đo quan trọng để đánh giá hiệu quả dài hạn của mô hình. Một giá trị cao cho thấy mô hình có khả năng sinh lời tốt trong khoảng thời gian dài, giúp các nhà đầu tư hoặc người dùng tự tin hơn khi áp dụng chiến lược vào thực tế. Một Return (Ann.) cao (ví dụ: trên 10-15%) được xem là tốt và cho thấy mô hình có khả năng tạo ra lợi nhuận đáng kể hàng năm. Ngược lại, một tỷ suất lợi nhuận thấp hoặc âm cho thấy mô hình hoạt động kém và không tạo ra giá trị tích cực. """
    context.append(results)
data['Context']=context
data.to_csv('prompt_data/new_return.csv', index=False)

## For MDD

In [46]:
import pandas as pd
import random
data = pd.read_csv('prompt_data/mdd.csv')

context = []
# Tạo các biến ngẫu nhiên
mdd_array = data['Value']
for i,_ in enumerate(mdd_array):
    max_drawdown = mdd_array[i]
    sharpe_ratio = round(random.uniform(-1, 3.5), 2)  
    volatility_ann =  round(random.uniform(5, 35), 2)  
    return_ann = round(sharpe_ratio*volatility_ann, 2)  
    sortino_ratio = round(random.uniform(0.5, 2.5), 2)  
    calmar_ratio = round((-1)*return_ann/max_drawdown, 2)
    win_rate = round(random.uniform(40, 70), 2)  
    avg_trade = round(random.uniform(-0.5, 2.5), 2) 
    generalization_score = round(random.uniform(-1, 1), 2) 
    recent_prediction_rate = random.randrange(0, 101, 10) 

    # Kết quả
    results = f"""Chỉ số của mô hình:
    Return (Ann.) [%]: {return_ann},
    Volatility (Ann.) [%]: {volatility_ann},
    Max. Drawdown [%]: {max_drawdown},
    Sharpe Ratio: {sharpe_ratio},
    Sortino Ratio: {sortino_ratio},
    Calmar Ratio: {calmar_ratio},
    Win Rate [%]: {win_rate},
    AvgTrade [%]: {avg_trade},
    Generalization Score: {generalization_score},
    Tỉ lệ dự đoán gần đây [%]: {recent_prediction_rate}.
    Các thông tin tài chính bổ sung: Maximum Drawdown (MDD) là mức sụt giảm lớn nhất từ đỉnh đến đáy của giá trị danh mục đầu tư hoặc tài sản trong một khoảng thời gian. Chỉ số này thể hiện rủi ro thua lỗ lớn nhất mà chiến lược có thể gặp phải. MDD giúp nhà đầu tư hiểu được mức giảm giá tồi tệ nhất có thể xảy ra, hỗ trợ trong việc quản lý rủi ro tổng thể của chiến lược. MDD thấp (ví dụ: dưới -10%) cho thấy rủi ro thua lỗ lớn nhất mà mô hình phải chịu không quá nghiêm trọng, biểu hiện khả năng bảo toàn vốn tốt. MDD cao (ví dụ: trên -30%) cho thấy rủi ro lớn, có thể không phù hợp với nhà đầu tư ưa thích chiến lược an toàn. """
    context.append(results)
data['Context']=context
data.to_csv('prompt_data/new_mdd.csv', index=False)

## For volatility

In [34]:
import pandas as pd
import random
data = pd.read_csv('prompt_data/vol.csv')

context = []
# Tạo các biến ngẫu nhiên
vol_array = data['Value']
for i,_ in enumerate(vol_array):
    volatility_ann = vol_array[i] 
    sharpe_ratio = round(random.uniform(-1, 3.5), 2)  
    return_ann = round(sharpe_ratio*volatility_ann, 2)  
    max_drawdown = round(random.uniform(-40, -10), 2)  
    sortino_ratio = round(random.uniform(0.5, 2.5), 2)  
    calmar_ratio = round((-1)*return_ann/max_drawdown, 2)
    win_rate = round(random.uniform(40, 70), 2)  
    avg_trade = round(random.uniform(-0.5, 2.5), 2) 
    generalization_score = round(random.uniform(-1, 1), 2) 
    recent_prediction_rate = random.randrange(0, 101, 10) 

    # Kết quả
    results = f"""Chỉ số của mô hình:
    Return (Ann.) [%]: {return_ann},
    Volatility (Ann.) [%]: {volatility_ann},
    Max. Drawdown [%]: {max_drawdown},
    Sharpe Ratio: {sharpe_ratio},
    Sortino Ratio: {sortino_ratio},
    Calmar Ratio: {calmar_ratio},
    Win Rate [%]: {win_rate},
    AvgTrade [%]: {avg_trade},
    Generalization Score: {generalization_score},
    Tỉ lệ dự đoán gần đây [%]: {recent_prediction_rate}.
    Các thông tin tài chính bổ sung: Volatility Ann [%] là thước đo sự dao động của lợi nhuận quanh giá trị trung bình trong một khoảng thời gian hàng năm. Chỉ số này cho thấy mức độ ổn định của mô hình dự đoán. Một độ biến động thấp biểu thị mô hình ổn định và ít rủi ro, trong khi một độ biến động cao cho thấy mô hình có sự dao động lớn hơn, có thể đi kèm với rủi ro lớn nhưng cũng mang lại cơ hội sinh lời cao hơn.
Volatility Ann [%] thấp (ví dụ: dưới 10-15%) cho thấy mô hình có sự ổn định cao và ít rủi ro, phù hợp với những chiến lược đầu tư an toàn.
Volatility Ann [%] cao (ví dụ: trên 20-30%) ám chỉ mô hình có độ dao động mạnh, đồng nghĩa với khả năng tạo ra lợi nhuận lớn hơn nhưng đi kèm với mức độ rủi ro cao hơn.
Chỉ số này giúp nhà đầu tư hiểu rõ mức độ biến động và rủi ro của một chiến lược hoặc mô hình giao dịch """
    context.append(results)
data['Context']=context
data.to_csv('prompt_data/new_vol.csv', index=False)

## For calmar

In [29]:
import pandas as pd
import random
data = pd.read_csv('prompt_data/calmar.csv')

context = []
# Tạo các biến ngẫu nhiên
cal_array = data['Value']
for i,_ in enumerate(cal_array):
    sharpe_ratio = round(random.uniform(-1, 3.5), 2)  
    volatility_ann =  round(random.uniform(5, 35), 2)  
    return_ann = round(sharpe_ratio*volatility_ann, 2)  
    calmar_ratio = cal_array[i]
    max_drawdown = round((-1)*return_ann/calmar_ratio, 2)  
    sortino_ratio = round(random.uniform(0.5, 2.5), 2)  
    win_rate = round(random.uniform(40, 70), 2)  
    avg_trade = round(random.uniform(-0.5, 2.5), 2) 
    generalization_score = round(random.uniform(-1, 1), 2) 
    recent_prediction_rate = random.randrange(0, 101, 10) 

    # Kết quả
    results = f"""Chỉ số của mô hình:
    Return (Ann.) [%]: {return_ann},
    Volatility (Ann.) [%]: {volatility_ann},
    Max. Drawdown [%]: {max_drawdown},
    Sharpe Ratio: {sharpe_ratio},
    Sortino Ratio: {sortino_ratio},
    Calmar Ratio: {calmar_ratio},
    Win Rate [%]: {win_rate},
    AvgTrade [%]: {avg_trade},
    Generalization Score: {generalization_score},
    Tỉ lệ dự đoán gần đây [%]: {recent_prediction_rate}.
    Các thông tin tài chính bổ sung: Calmar Ratio đo lường tỷ lệ giữa lợi nhuận hàng năm và mức sụt giảm lớn nhất (maximum drawdown) mà chiến lược phải trải qua. Công thức tính là 
Calmar Ratio = Return (Ann.) / Maximum Drawdown. Chỉ số này càng cao chứng tỏ mô hình có khả năng tạo ra lợi nhuận mà không phải chịu rủi ro lớn, là thước đo quan trọng cho chiến lược đầu tư dài hạn. Calmar Ratio trên 0.5-1 được coi là tốt và trên 2 là rất tích cực, chỉ ra mô hình có thể sinh lời cao so với rủi ro thua lỗ lớn nhất. Chỉ số thấp dưới 0.5 cảnh báo rằng lợi nhuận không bù đắp đủ cho rủi ro. """
    context.append(results)
data['Context']=context
data.to_csv('prompt_data/new_calmar.csv', index=False)

## For win rate

In [41]:
import pandas as pd
import random
data = pd.read_csv('prompt_data/winrate.csv')

context = []
# Tạo các biến ngẫu nhiên
win_array = data['Value']
for i,_ in enumerate(cal_array):
    sharpe_ratio = round(random.uniform(-1, 3.5), 2)  
    volatility_ann =  round(random.uniform(5, 35), 2)  
    return_ann = round(sharpe_ratio*volatility_ann, 2)  
    max_drawdown = round(random.uniform(-40, -10), 2)  
    calmar_ratio =  round((-1)*return_ann/max_drawdown, 2)
    sortino_ratio = round(random.uniform(0.5, 2.5), 2)  
    win_rate = win_array[i]
    avg_trade = round(random.uniform(-0.5, 2.5), 2) 
    generalization_score = round(random.uniform(-1, 1), 2) 
    recent_prediction_rate = random.randrange(0, 101, 10) 

    # Kết quả
    results = f"""Chỉ số của mô hình:
    Return (Ann.) [%]: {return_ann},
    Volatility (Ann.) [%]: {volatility_ann},
    Max. Drawdown [%]: {max_drawdown},
    Sharpe Ratio: {sharpe_ratio},
    Sortino Ratio: {sortino_ratio},
    Calmar Ratio: {calmar_ratio},
    Win Rate [%]: {win_rate},
    AvgTrade [%]: {avg_trade},
    Generalization Score: {generalization_score},
    Tỉ lệ dự đoán gần đây [%]: {recent_prediction_rate}.
    Các thông tin tài chính bổ sung: Win Rate là tỷ lệ phần trăm số giao dịch chính xác trong tổng số giao dịch mà mô hình thực hiện. Đây là một chỉ số phổ biến để đánh giá tần suất mô hình đưa ra các dự đoán chính xác. Một tỷ lệ cao cho thấy mô hình có khả năng chính xác trong hầu hết các trường hợp, làm tăng độ tin cậy vào chiến lược. Win Rate trên 50% cho thấy mô hình thắng nhiều hơn thua, là dấu hiệu tích cực. Win Rate trên 60-70% thường được coi là rất tốt. Một Win Rate thấp hơn 50% có thể cho thấy mô hình cần cải thiện, trừ khi các giao dịch thắng có lợi nhuận lớn bù đắp được các lần thua."""
    context.append(results)
data['Context']=context
data.to_csv('prompt_data/new_winrate.csv', index=False)

##  For Avg Trade

In [43]:
import pandas as pd
import random
data = pd.read_csv('prompt_data/avgtrade.csv')

context = []
# Tạo các biến ngẫu nhiên
avgtrade_array = data['Value']
for i,_ in enumerate(avgtrade_array):
    sharpe_ratio = round(random.uniform(-1, 3.5), 2)  
    volatility_ann =  round(random.uniform(5, 35), 2)  
    return_ann = round(sharpe_ratio*volatility_ann, 2)  
    max_drawdown = round(random.uniform(-40, -10), 2)  
    calmar_ratio =  round((-1)*return_ann/max_drawdown, 2)
    sortino_ratio = round(random.uniform(0.5, 2.5), 2)  
    win_rate =  round(random.uniform(40, 70), 2)  
    avg_trade = avgtrade_array[i]
    generalization_score = round(random.uniform(-1, 1), 2) 
    recent_prediction_rate = random.randrange(0, 101, 10) 

    # Kết quả
    results = f"""Chỉ số của mô hình:
    Return (Ann.) [%]: {return_ann},
    Volatility (Ann.) [%]: {volatility_ann},
    Max. Drawdown [%]: {max_drawdown},
    Sharpe Ratio: {sharpe_ratio},
    Sortino Ratio: {sortino_ratio},
    Calmar Ratio: {calmar_ratio},
    Win Rate [%]: {win_rate},
    AvgTrade [%]: {avg_trade},
    Generalization Score: {generalization_score},
    Tỉ lệ dự đoán gần đây [%]: {recent_prediction_rate}.
    Các thông tin tài chính bổ sung: AvgTrade thể hiện mức lợi nhuận trung bình trên mỗi giao dịch. Đây là chỉ số đo lường mức hiệu quả của từng giao dịch mà mô hình thực hiện. Chỉ số này giúp hiểu rõ hơn về lợi nhuận mà mỗi lần quyết định mang lại, cho thấy liệu chiến lược có mang lại giá trị thực tế trong từng bước hành động hay không. AvgTrade dương và lớn (ví dụ trên 1.5%) cho thấy mỗi giao dịch trung bình đem lại lợi nhuận tốt. Nếu AvgTrade quá thấp hoặc âm, mô hình có thể không hiệu quả trong việc tạo lợi nhuận ổn định trên từng giao dịch."""
    context.append(results)
data['Context']=context
data.to_csv('prompt_data/new_avgtrade.csv', index=False)

## For recent

In [49]:
import pandas as pd
import random
data = pd.read_csv('prompt_data/recent.csv')

context = []
# Tạo các biến ngẫu nhiên
recent_array = data['Value']
for i,_ in enumerate(recent_array):
    sharpe_ratio = round(random.uniform(-1, 3.5), 2)  
    volatility_ann =  round(random.uniform(5, 35), 2)  
    return_ann = round(sharpe_ratio*volatility_ann, 2)  
    max_drawdown = round(random.uniform(-40, -10), 2)  
    calmar_ratio =  round((-1)*return_ann/max_drawdown, 2)
    sortino_ratio = round(random.uniform(0.5, 2.5), 2)  
    win_rate =  round(random.uniform(40, 70), 2)  
    avg_trade = round(random.uniform(-0.5, 2.5), 2) 
    generalization_score = round(random.uniform(-1, 1), 2) 
    recent_prediction_rate = recent_array[i]

    # Kết quả
    results = f"""Chỉ số của mô hình:
    Return (Ann.) [%]: {return_ann},
    Volatility (Ann.) [%]: {volatility_ann},
    Max. Drawdown [%]: {max_drawdown},
    Sharpe Ratio: {sharpe_ratio},
    Sortino Ratio: {sortino_ratio},
    Calmar Ratio: {calmar_ratio},
    Win Rate [%]: {win_rate},
    AvgTrade [%]: {avg_trade},
    Generalization Score: {generalization_score},
    Tỉ lệ dự đoán gần đây [%]: {recent_prediction_rate}.
    Tỉ lệ dự đoán gần đây là chỉ số thể hiện mức độ chính xác của các dự đoán gần nhất mà mô hình đã đưa ra. Chỉ số này giúp đánh giá nhanh hiệu quả hiện tại của mô hình và xác định xem có cần điều chỉnh hoặc cập nhật gì không để đảm bảo mô hình tiếp tục hoạt động tốt trong ngắn hạn. Một tỷ lệ dự đoán chính xác gần đây cao (ví dụ: trên 70-80%) thể hiện mô hình đang hoạt động tốt trong ngắn hạn. Nếu tỷ lệ này thấp hơn 50%, mô hình có thể đang gặp vấn đề cần điều chỉnh hoặc cập nhật để cải thiện hiệu suất. """
    context.append(results)
data['Context']=context
data.to_csv('prompt_data/new_recent.csv', index=False)

## For gs

In [7]:
import pandas as pd
import random
data = pd.read_csv('prompt_data/gs.csv')

context = []
# Tạo các biến ngẫu nhiên
gs_array = data['Value']
for i,_ in enumerate(gs_array):
    sharpe_ratio = round(random.uniform(-1, 3.5), 2)  
    volatility_ann =  round(random.uniform(5, 35), 2)  
    return_ann = round(sharpe_ratio*volatility_ann, 2)  
    max_drawdown = round(random.uniform(-40, -10), 2)  
    calmar_ratio =  round((-1)*return_ann/max_drawdown, 2)
    sortino_ratio = round(random.uniform(0.5, 2.5), 2)  
    win_rate =  round(random.uniform(40, 70), 2)  
    avg_trade = round(random.uniform(-0.5, 2.5), 2) 
    generalization_score = gs_array[i]
    recent_prediction_rate = random.randrange(0, 101, 10) 

    # Kết quả
    results = f"""Chỉ số của mô hình:
    Return (Ann.) [%]: {return_ann},
    Volatility (Ann.) [%]: {volatility_ann},
    Max. Drawdown [%]: {max_drawdown},
    Sharpe Ratio: {sharpe_ratio},
    Sortino Ratio: {sortino_ratio},
    Calmar Ratio: {calmar_ratio},
    Win Rate [%]: {win_rate},
    AvgTrade [%]: {avg_trade},
    Generalization Score: {generalization_score},
    Tỉ lệ dự đoán gần đây [%]: {recent_prediction_rate}.
    Generalization Score đo khả năng của mô hình trong việc hoạt động hiệu quả trên dữ liệu mới, chưa từng gặp trong quá trình huấn luyện, người dùng có thể dùng chỉ số này để xem xét liệu mô hình có tin cậy không. Điểm số này giúp xác định liệu mô hình có bị quá khớp (overfitting) với dữ liệu huấn luyện hay không. Nó được tính  bằng cách lấy kết quả trên tập train chia cho tập validation và trừ đi 1. Generalization Score càng gần 0 chứng tỏ mô hình càng ít bị overfit và hoạt động càng tốt trên dữ liệu mới. """
    context.append(results)
data['Context']=context
data.to_csv('prompt_data/new_gs.csv', index=False)

## Mixing metrics

In [17]:
import random
import openai
import pandas as pd

# Danh sách chỉ số tài chính có thể được sử dụng
financial_metrics = [
    "Sharpe Ratio", "Volatility Ann", "Return Ann", "Max Drawdown",
    "Calmar Ratio", "Sortino Ratio", "Win Rate", "Avg Trade", 
    "Generalization Score", "Recent Prediction Rate"
]

def generate_random_metrics():
    """Tạo một từ điển chứa các chỉ số ngẫu nhiên và giá trị của chúng."""
    metrics = {
        "Sharpe Ratio": round(random.uniform(-1, 3.5), 2),
        "Volatility Ann": round(random.uniform(5, 35), 2),
        "Return Ann": lambda sharpe_ratio, volatility: round(sharpe_ratio * volatility, 2),
        "Max Drawdown": round(random.uniform(-40, -10), 2),
        "Calmar Ratio": lambda return_ann, max_drawdown: round((-1) * return_ann / max_drawdown, 2),
        "Sortino Ratio": round(random.uniform(0.5, 2.5), 2),
        "Win Rate": round(random.uniform(40, 70), 2),
        "Avg Trade": round(random.uniform(-0.5, 2.5), 2),
        "Generalization Score": round(random.uniform(-1, 1), 2),
        "Recent Prediction Rate": random.randrange(0, 101, 10)
    }
    return metrics

def evaluate_financial_scenario():
    scenarios = []
    
    for i in range(1, 150):
        # Chọn ngẫu nhiên số lượng chỉ số để hỏi (từ 2 đến toàn bộ)
        num_metrics = random.randint(2, len(financial_metrics))
        selected_metrics = random.sample(financial_metrics, num_metrics)
        
        # Tạo giá trị ngẫu nhiên cho các chỉ số
        metric_values = generate_random_metrics()
        
        # Xử lý các chỉ số phụ thuộc
        sharpe_ratio = metric_values["Sharpe Ratio"]
        volatility_ann = metric_values["Volatility Ann"]
        metric_values["Return Ann"] = metric_values["Return Ann"](sharpe_ratio, volatility_ann)
        max_drawdown = metric_values["Max Drawdown"]
        metric_values["Calmar Ratio"] = metric_values["Calmar Ratio"](metric_values["Return Ann"], max_drawdown)
        
        # Xây dựng prompt chỉ với các chỉ số đã chọn
        metrics_text = "\n".join([f"- {metric}: {metric_values[metric]}" for metric in selected_metrics])
        prompt = f"""Giả sử bạn đang đánh giá một mô hình tài chính với các chỉ số sau:\n{metrics_text}\n
        nếu có chỉ số Generalization Score, thì nó càng gần 0 càng tốt, nhưng đừng thêm thông tin này vào câu hỏi. Hãy đặt các câu hỏi và câu trả lời xung quanh tất cả chỉ số này cùng một lúc, không làm riêng lẻ 1 cái. Trả về kết quả theo định dạng sau bằng tiếng Việt:
        'Question:...
         Choose:...
         Reject:...'"""

        try:
            response = openai.ChatCompletion.create(
                model="gpt-4o-2024-08-06",
                messages=[
                    {"role": "system", "content": "You are a financial analyst."},
                    {"role": "user", "content": prompt}
                ],
                max_tokens=200,
                temperature=0.8
            )
            
            answer = response['choices'][0]['message']['content'].strip()
            scenarios.append((answer, {metric: metric_values[metric] for metric in selected_metrics}))
        
        except Exception as e:
            print(f"Error occurred: {e}")
            scenarios.append(("Error", {metric: metric_values[metric] for metric in selected_metrics}))

    return scenarios

# Lấy các tình huống và tạo DataFrame
results = evaluate_financial_scenario()
data = []

for entry, metric_values in results:
    questions = entry.split('Question:')[1:]
    for question in questions:
        try:
            question = question.strip()
            choose = question.split('Choose:')[1].split('Reject:')[0].strip()
            reject = question.split('Reject:')[1].strip()
            question_text = question.split('\n')[0].strip()

            # Thêm vào dữ liệu
            row = [question_text, choose, reject] + [metric_values.get(metric, '') for metric in financial_metrics]
            data.append(row)
        except IndexError:
            print(f"Warning: Entry không đúng định dạng:\n{question}")
            continue

# Tạo DataFrame và lưu vào file CSV
df = pd.DataFrame(data, columns=['Câu hỏi mẫu', 'Câu trả lời nên chọn', 'Câu trả lời nên từ chối'] + financial_metrics)
df.to_csv('financial_scenarios4.csv', index=False)


Dựa trên các chỉ số đã cho, bạn có thể đánh giá rủi ro và lợi nhuận tiềm năng của mô hình tài chính này như thế nào?

Choose: Dựa vào các chỉ số:
- Avg Trade: 2.31 cho thấy rằng trung bình mỗi giao dịch có lợi nhuận dương, tuy nhiên con số này không đủ để bù đắp cho tỷ lệ lợi nhuận âm hàng năm.
- Win Rate: 57.27 cho thấy tỷ lệ giao dịch thắng là khá cao, tuy nhiên tỷ lệ này cần được xem xét trong bối cảnh của tỷ lệ lợi nhuận âm hàng năm và mức giảm tối đa.
- Max Drawdown: -36.12 cho thấy mức sụt giảm lớn nhất từ đỉnh đến đáy, điều này thể hiện rủi ro đáng kể.
- Return Ann: -9.45 cho thấy mô hình đang lỗ hàng năm, điều này không khuyến khích cho
Dựa trên các chỉ số tài chính đã đưa ra, mô hình tài chính này có thể được xem là có hiệu quả như thế nào trong việc quản lý rủi ro và tối ưu hóa lợi nhuận?

Choose: Với Return Ann là 11.16% và Volatility Ann là 34.86%, mô hình này có khả năng tạo ra lợi nhuận nhưng đi kèm với mức độ biến động khá cao. Sharpe Ratio ở mức 0.32 cho thấy lợi nhuận 

In [25]:
import pandas as pd
import random

# Đọc dữ liệu từ file CSV đã tạo
data = pd.read_csv('financial_scenarios4.csv')

# Hàm để lấy giá trị từ dòng hiện tại hoặc tạo giá trị mặc định ngẫu nhiên
def generate_default_metrics(row):
    # Kiểm tra giá trị trong row; nếu không có giá trị thì sinh giá trị ngẫu nhiên
    if not pd.isna(row['Sharpe Ratio']):
        sharpe_ratio = row['Sharpe Ratio']
    elif not pd.isna(row['Volatility Ann']) and not pd.isna(row['Return Ann']):
        sharpe_ratio = round(row['Return Ann'] / row['Volatility Ann'],2)
    else:
        sharpe_ratio = round(random.uniform(-1, 3.5), 2)
    
    if not pd.isna(row['Volatility Ann']):
        volatility_ann = row['Volatility Ann']
    elif not pd.isna(row['Return Ann']) and not pd.isna(sharpe_ratio):
        volatility_ann = round(row['Return Ann'] / sharpe_ratio,2)
    else:
        volatility_ann = round(random.uniform(5, 35), 2)
    
    if not pd.isna(row['Return Ann']):
        return_ann = row['Return Ann']
    elif not pd.isna(sharpe_ratio) and not pd.isna(volatility_ann):
        return_ann = round(sharpe_ratio * volatility_ann,2)
    else:
        return_ann = round(sharpe_ratio * volatility_ann, 2)
    
    if not pd.isna(row['Max Drawdown']):
        max_drawdown = row['Max Drawdown']
    elif not pd.isna(return_ann) and not pd.isna(row['Calmar Ratio']):
        max_drawdown = round((-1) * return_ann / row['Calmar Ratio'], 2)
    else:
        max_drawdown = round(random.uniform(-40, -10), 2)
    
    if not pd.isna(row['Calmar Ratio']):
        calmar_ratio = row['Calmar Ratio']
    elif not pd.isna(return_ann) and not pd.isna(max_drawdown):
        calmar_ratio = round((-1) * return_ann / max_drawdown, 2)
    else:
        calmar_ratio = round((-1) * return_ann / max_drawdown, 2)
    
    sortino_ratio = row['Sortino Ratio'] if not pd.isna(row['Sortino Ratio']) else round(random.uniform(0.5, 2.5), 2)
    win_rate = row['Win Rate'] if not pd.isna(row['Win Rate']) else round(random.uniform(40, 70), 2)
    avg_trade = row['Avg Trade'] if not pd.isna(row['Avg Trade']) else round(random.uniform(-0.5, 2.5), 2)
    generalization_score = row['Generalization Score'] if not pd.isna(row['Generalization Score']) else round(random.uniform(-1, 1), 2)
    recent_prediction_rate = row['Recent Prediction Rate'] if not pd.isna(row['Recent Prediction Rate']) else random.randrange(0, 101, 10)
    
    return {
        "Sharpe Ratio": sharpe_ratio,
        "Volatility Ann": volatility_ann,
        "Return Ann": return_ann,
        "Max Drawdown": max_drawdown,
        "Calmar Ratio": calmar_ratio,
        "Sortino Ratio": sortino_ratio,
        "Win Rate": win_rate,
        "Avg Trade": avg_trade,
        "Generalization Score": generalization_score,
        "Recent Prediction Rate": recent_prediction_rate
    }

# Duyệt từng hàng trong dữ liệu và tạo context
context = []
for index, row in data.iterrows():
    # Lấy các chỉ số (có hoặc ngẫu nhiên) cho từng dòng
    metrics = generate_default_metrics(row)
    
    # Tạo context cho dòng hiện tại với các chỉ số đầy đủ
    results = f"""Chỉ số của mô hình:
    Return (Ann.) [%]: {metrics['Return Ann']},
    Volatility (Ann.) [%]: {metrics['Volatility Ann']},
    Max. Drawdown [%]: {metrics['Max Drawdown']},
    Sharpe Ratio: {metrics['Sharpe Ratio']},
    Sortino Ratio: {metrics['Sortino Ratio']},
    Calmar Ratio: {metrics['Calmar Ratio']},
    Win Rate [%]: {metrics['Win Rate']},
    AvgTrade [%]: {metrics['Avg Trade']},
    Generalization Score: {metrics['Generalization Score']},
    Tỉ lệ dự đoán gần đây [%]: {metrics['Recent Prediction Rate']}.
    """
    context.append(results)

# Gán cột 'Context' vào DataFrame và lưu lại
data['Context'] = context
data.to_csv('prompt_data/new_mixing.csv', index=False)

## For QA data

In [5]:
import pandas as pd
import random
data = pd.read_csv('prompt_data/qa.csv')

context = []
# Tạo các biến ngẫu nhiên

for i in range(data.shape[0]):
    sharpe_ratio = round(random.uniform(-1, 3.5), 2)  
    volatility_ann =  round(random.uniform(5, 35), 2)  
    return_ann = round(sharpe_ratio*volatility_ann, 2)  
    max_drawdown = round(random.uniform(-40, -10), 2)  
    calmar_ratio =  round((-1)*return_ann/max_drawdown, 2)
    sortino_ratio = round(random.uniform(0.5, 2.5), 2)  
    win_rate =  round(random.uniform(40, 70), 2)  
    avg_trade = round(random.uniform(-0.5, 2.5), 2) 
    generalization_score = round(random.uniform(-1, 1), 2) 
    recent_prediction_rate = random.randrange(0, 101, 10) 

    # Kết quả
    results = f"""Chỉ số của mô hình:
    Return (Ann.) [%]: {return_ann},
    Volatility (Ann.) [%]: {volatility_ann},
    Max. Drawdown [%]: {max_drawdown},
    Sharpe Ratio: {sharpe_ratio},
    Sortino Ratio: {sortino_ratio},
    Calmar Ratio: {calmar_ratio},
    Win Rate [%]: {win_rate},
    AvgTrade [%]: {avg_trade},
    Generalization Score: {generalization_score},
    Tỉ lệ dự đoán gần đây [%]: {recent_prediction_rate}. """
    context.append(results)
data['Context']=context
data.to_csv('prompt_data/new_qa.csv', index=False)

In [3]:
data.shape[0]

1961

# Combine Data

In [6]:
from datasets import Dataset, DatasetDict
import pandas as pd
import glob
from sklearn.model_selection import train_test_split

# Đọc và kết hợp các file CSV
file_paths = glob.glob("D:/Study file/Fall 2024 - Capstone/Capstone/orpo_data/*.csv")
df_list = [pd.read_csv(file) for file in file_paths]
combined_df = pd.concat(df_list, ignore_index=True)

# Chia dữ liệu thành 85% train và 15% test
train_df, test_df = train_test_split(combined_df, test_size=0.15, random_state=42)

train_df = train_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)


# Chuyển đổi DataFrames sang Dataset của Hugging Face
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

# Tạo DatasetDict
dataset = DatasetDict({
    'train': train_dataset,
    'test': test_dataset
})

# Lưu DatasetDict thành file Arrow
dataset.save_to_disk("combined_dataset")


d:\Study file\Fall 2024 - Capstone\Capstone\.conda\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Saving the dataset (1/1 shards): 100%|██████████| 516/516 [00:00<00:00, 114912.44 examples/s]


In [7]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Instruction', 'Context', 'Text', 'Choose', 'Reject'],
        num_rows: 2922
    })
    test: Dataset({
        features: ['Instruction', 'Context', 'Text', 'Choose', 'Reject'],
        num_rows: 516
    })
})